In [98]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import time
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.palettes import BuPu, Greens, Set3, Blues, Oranges
from scipy.fftpack import fft, rfft
import math
import datetime
import uuid
import json
from copy import deepcopy

In [12]:
output_notebook()
opts = dict(plot_width=800, plot_height=400, min_border=0)

Loading BokehJS ...

In [13]:
def countDown():
    print 'Recording Audio for 2 seconds'
    for i in range(3, 0, -1):
        time.sleep(1)
        print i
    print 'Start'

In [14]:
dataTemplate = {
  "aClip": {
    "info" : {
      "id" : None,
      "type" : None,
      "author" : None,
      "timeStamp" : None,
      "output" : None,
    },
    "t": {
      "tData": None,
      "tFs" : None,
      "tLen" : None,
    },
    "f" : {
      "fData" : None,
      "fFreq" : None,
      "fLen" : None,
    }
  }
}

In [15]:
fs=1024
duration = 2  #In seconds

In [16]:
# Some values don't want to save
# json.dump(toSave, outfile, cls=MyEncoder)
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

In [17]:
#Record single - returns audio clip
def recordSingle():
    countDown()

    #Record
    myRecording1 = sd.rec(duration * fs, samplerate=fs, channels=1)
    sd.wait()

    #Play
    print "Audio recording complete , Play Audio"
    sd.play(myRecording1, fs)
    sd.wait()
    
    #Each sample is of type list. Convert from list to int
    bClip = []
    for i in range(len(myRecording1)):
        bClip.append( myRecording1[i][0] )
    
    return bClip

In [18]:
#Record multiple clips at once - returns list 
def recordMultiple(numberOfSamples):
    myRecordings = []
    
    for i in range(numberOfSamples):
        myRecordings.append( recordSingle() )
    
    return myRecordings   

In [19]:
#Plot Audio Clips
def plotAudioClips(_aClips):
    p2 = figure(**opts)

    for i in range(len(aClips)):
        p2.line(range(len(aClips[i])), aClips[i], color=Set3[9][i], line_width=1)

    #show(p2)

In [24]:
def getFFT(_timeSeries, displayPlot):

    aClips = _timeSeries
    _displayPlot = displayPlot
    
    #Create fft
    p3 = figure(**opts)

    ffts = []
    for i in range(len(aClips)):

        #Get data
        data = aClips[i]

        #Smooth the FFT by windowing data
        data = data * np.hanning(len(data))

        #Preform FFT and take only real data
        fft = abs(np.fft.fft(data).real)

        #Keep only first half
        fft = fft[:int(len(fft))/2]

        #Create Frequency list
        freq = np.fft.fftfreq( (fs * duration) ,1.0/fs)

        #Keep only first half
        freq = freq[:int(len(freq))/2] 

        #At this point, the length of the fft/freq is cut in reduced from 2048 to 1024. But the cut off frequency is actually 512.
        #The freq axis shows why. Values are now 0, 0.5, 1, 1.5Hz...
        #Therefore drop every other value
        fft = fft[0::2]
        freq = freq[0::2]

        #Save for later
        ffts.append( fft )
        
        #Trim off what isn't needed
        maxList = np.sort(fft[100:-100])

        #Find peak
        freqPeak = np.where(fft==maxList[-1])[0]
        #print("peak frequency: %d Hz"%freqPeak)

        #For plotting patches, we must close the polygon
        fftPlot = np.append(fft, [fft[-1], 0] )
        freqPlot = np.append(freq, [len(freq)+1, len(freq)+1])

        legendName = 'Data' + str(i)

        #Plot
        #if i < 3:
        p3.patches([freqPlot], [fftPlot], line_width=1, color=Oranges[9][i], alpha=0.1, legend=legendName)
        #else:
            #p3.patches([freqPlot], [fftPlot], line_width=1, color=Blues[9][i-3], alpha=0.1, legend=legendName)

    if _displayPlot:
        p3.legend.location = 'top_right'
        p3.legend.click_policy = 'hide'
        show(p3)
        pass
    
    return ffts, freq, p3

In [107]:
#Record and plot multiple
aClips = recordMultiple(6)

plotAudioClips(aClips)

ffts, freq, plot3 = getFFT(aClips, displayPlot=True)

show(plot3)


Recording Audio for 2 seconds
3
2
1
Start
Audio recording complete , Play Audio
Recording Audio for 2 seconds
3
2
1
Start
Audio recording complete , Play Audio
Recording Audio for 2 seconds
3
2
1
Start
Audio recording complete , Play Audio
Recording Audio for 2 seconds
3
2
1
Start
Audio recording complete , Play Audio
Recording Audio for 2 seconds
3
2
1
Start
Audio recording complete , Play Audio
Recording Audio for 2 seconds
3
2
1
Start
Audio recording complete , Play Audio


In [108]:
#Format data

dType = 'meow'

if dType == 'woof':
    dOutput = [1.0, 0.0]
elif dType == 'meow':
    dOutput = [0.0, 1.0]
    
dAuthor = 'diego'

toSave = []

for i in range(len(ffts)):

    dataT = dataTemplate

    dataT['aClip']['info']['id'] = str(uuid.uuid4())   
    dataT['aClip']['info']['type'] = dType
    dataT['aClip']['info']['author'] = dAuthor
    dataT['aClip']['info']['timeStamp'] = str( datetime.datetime.now() )
    dataT['aClip']['info']['output'] = dOutput

    dataT['aClip']['t']['tData'] = aClips[i]
    dataT['aClip']['t']['tFs'] = fs
    dataT['aClip']['t']['tLen'] = len(aClips[i])

    dataT['aClip']['f']['fData'] = ffts[i].tolist()
    dataT['aClip']['f']['fFreq'] = freq.tolist()
    dataT['aClip']['f']['fLen'] = len(ffts[i])

    
    toSave.append( copy.deepcopy(dataT) )
    
print 'Data formated - Ready to save' 


Data formated - Ready to save


In [109]:
#Save data - Appends to file

#FileName of output file
fileName = 'meow1.json'

with open(fileName, 'w') as outfile:
    json.dump(toSave, outfile, cls=MyEncoder)